In [47]:
#from sklearn.neighbors import NearestNeighbors
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pa
import json as js
import networkx as nx
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt

In [48]:
file = open('export.json',)
data = js.load(file)
elements = data["elements"]
file.close()

In [49]:
def haversine(point1, point2):
    point1[0], point1[1], point2[0], point2[1] = map(radians, [point1[0], point1[1], point2[0], point2[1]])

    # haversine formula 
    dlon = point2[0] - point1[0] 
    dlat = point2[1] - point1[1] 
    a = sin(dlat/2)**2 + cos(point1[1]) * cos(point2[1]) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [50]:
#regroupe les coordonnées en couple
nodesMap = {}
for e in elements:
    if (e["type"] == "node"):
        if("tags" in e):
            if("name" in e["tags"]):
                if(e["tags"]["name"].startswith("Aire")):
                    print(e["id"],e["tags"]["name"])
                    point = [e["lat"],e["lon"],1]
                else:
                    point = [e["lat"],e["lon"],0]
            else:
                point = [e["lat"],e["lon"],0]

        else:
            point = [e["lat"],e["lon"],0]
        # ajout dans le dictionnaire
        nodesMap.update({e["id"] : point})

print(nodesMap[1092718161])


21462872 Aire de Meillac
21463090 Aire de Bordeaux-Cestas
36133148 Aire de Cocumont
101439017 Aire de Lugos-Est
244598847 Aire d'Aquitaine Nord
245469652 Aire de Thouars
245469945 Aire de Thouars
249778863 Aire des Vignes Nord
255725311 Aire de Saugon
257156240 Aire des PalombiÃ¨res
313750636 Aire de la Gare de PÃ©age d'Arveyres
313753580 Aire des PalombiÃ¨res
315104278 Aire des Vignes Sud
315104887 Aire de la Gare de PÃ©age d'Arveyres
318151675 Aire des Terres de Graves
318182389 Aire des Terres de Graves
458755211 Aire de Cocumont
513717312 Aire de Bordeaux-Cestas
726554853 Aire de l'Estalot
736942879 Aire de Lugos-Ouest
778654326 Aire du Bazadais
987039211 Aire du Bazadais
1091814886 Aire de BÃ©denac Ouest
1876595132 Aire de Saint-Caprais
1893894292 Aire de CÃ©zac
2587088569 Aire de Boisredon
5761204260 Aire de Saugon
[44.2812476, -0.2325377, 0]


In [51]:
# parcourir les types "way"
# noSimpleGraph : {idNode : {
#        nextNodes : [node1,node2,...],
#        prevNodes : [node1,node2,...]
#}}
# 
noSimpleGraph = {}

def updateGraph(idNode,nextNode,prevNode):
    
    if nextNode != None:
        noSimpleGraph[idNode]["nextNodes"].append(nextNode)
    if prevNode != None:
        noSimpleGraph[idNode]["prevNodes"].append(prevNode)


for nodes in elements:
    if (nodes["type"] == "way"):
        for i in range(len(nodes["nodes"])):
            idNode = nodes["nodes"][i] 

            if idNode not in noSimpleGraph:
                noSimpleGraph.update({idNode : {"nextNodes" :[],
                                       "prevNodes" : []}})

            if i == 0:
               updateGraph(idNode,nodes["nodes"][i+1],None)

            elif i == (len(nodes["nodes"])-1):
                updateGraph(idNode,None,nodes["nodes"][i-1])
            else:
                updateGraph(idNode,nodes["nodes"][i+1],nodes["nodes"][i-1])
            
#print(noSimpleGraph)

print(noSimpleGraph[29557670]["nextNodes"])


[1693220385, 5895826612]


In [56]:
def distanceAvecProchaineAire(aireDep,dist):
    voisins = noSimpleGraph[aireDep]["nextNodes"]
    print(nodesMap[voisins])
    for voisin in voisins:
        if voisin is None :
            #print("error : no more Node")
            return 0,None
        if(nodesMap[voisin][2] == 1):
            return dist,voisin
        dist = dist + haversine([nodesMap[aireDep][0],nodesMap[aireDep][1]],[nodesMap[voisin][0],nodesMap[voisin][1]])
        return distanceAvecProchaineAire(voisin,dist)

dist = distanceAvecProchaineAire(1449891170,0)
print(dist)

TypeError: unhashable type: 'list'

In [58]:
def stationsOnWay(graph, way, stations):
    visited = set() # Créez un ensemble pour stocker les nœuds visités
    stack = [way[0]] # Créez une pile pour stocker les nœuds à visiter
    print(f"Chemin initialisé au noeud {way[0]}.") # Affichez le nœud de départ

    while stack: # Tant que la pile n'est pas vide
            current_node = stack.pop()  # Nœud en haut de la pile
            visited.add(current_node)  # Marquer le nœud comme visité

            # Vérifier si le nœud courant est dans la liste des stations
            if current_node in stations:
                print(f"La station {current_node} se trouve sur le chemin.")

            # Si le nœud courant est le nœud de fin, recherche terminée
            if current_node == way[1]:
                print(f"Noeud {way[1]} atteint, chemin parcouru.")
                break

            # Ajouter les nœuds voisins non visités à la pile
            for next_node in graph[current_node]['nextNodes']:
                if next_node not in visited:
                    stack.append(next_node)


graph = noSimpleGraph
way = [9576155790, 7509397404]
stations = [node_id for node_id, data in nodesMap.items() if data[2] == 1]

stationsOnWay(graph, way, stations)



Chemin initialisé au noeud 9576155790.
La station 21462872 se trouve sur le chemin.
Noeud 7509397404 atteint, chemin parcouru.
